In [1]:
#Importaciones basicas
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
# graficos incrustados
%matplotlib inline

#Carga de los CSV
BBVA_data = pd.read_csv('train_clientes.csv', header=0)
print("Data Train: ", BBVA_data.shape)
BBVA_test = pd.read_csv('test_clientes.csv', header=0)
print("Data Test:  ", BBVA_test.shape)

Data Train:  (70000, 53)
Data Test:   (30000, 52)


In [2]:
if BBVA_data.isnull().any().any():
    BBVA_data = BBVA_data.dropna()
    print("Clean Data Train: ", BBVA_data.shape)
    

Clean Data Train:  (56262, 53)


In [3]:
if BBVA_test.isnull().any().any():
    tipos = BBVA_test.columns.to_series().groupby(BBVA_test.dtypes).groups
    ctext = tipos[np.dtype('object')]
    columnas = BBVA_test.columns  # lista de todas las columnas
    cnum = list(set(columnas) - set(ctext))
    #COmpletando los valores nulos de colummnas numericas con la media
    for c in cnum:
        mean = BBVA_test[c].mean()
        BBVA_test[c] = BBVA_test[c].fillna(mean)
    # Completando valores nulos de columnas categóricas con la moda
    for c in ctext:
        mode = BBVA_test[c].mode()[0]
        BBVA_test[c] = BBVA_test[c].fillna(mode)
    print("Clean Data Test: ", BBVA_test.shape, BBVA_test.isnull().any().any())
    

Clean Data Test:  (30000, 52) False


In [4]:
def factorizacion_data(datus):
    #Pasando los datos de edad a rango de 10 años
    EDAD2 = pd.cut(datus['EDAD'], range(0, 150, 10))
    datus['EDAD'] = EDAD2
    
    #Factorizando la columna EDAD
    datus['EDAD'], _ = pd.factorize(datus['EDAD'])
    
    #Factorizar las demas columnas de datos categoricos
    tipos = datus.columns.to_series().groupby(datus.dtypes).groups
    ctext = tipos[np.dtype('object')]
    for c in ctext:
        datus[c], _ = pd.factorize(datus[c])
    print(type(datus), datus.shape)
    return datus

BBVA_data = factorizacion_data(BBVA_data)
BBVA_test = factorizacion_data(BBVA_test)

<class 'pandas.core.frame.DataFrame'> (56262, 53)
<class 'pandas.core.frame.DataFrame'> (30000, 52)


In [5]:
print(BBVA_data.isnull().any().any())
print(BBVA_test.isnull().any().any())

False
False


In [6]:
X_train = BBVA_data.drop(['ID_CORRELATIVO', 'ATTRITION'], axis=1)
print(type(X_train), X_train.shape)
X_predic = BBVA_test.drop(['ID_CORRELATIVO'], axis=1)
print(type(X_predic), X_predic.shape)
y_train = BBVA_data['ATTRITION'].values
print(type(y_train), y_train.shape)

<class 'pandas.core.frame.DataFrame'> (56262, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)
<class 'numpy.ndarray'> (56262,)


In [8]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import RFE

columnas = list(X_train.columns.values)
era = RFE(ExtraTreesClassifier(), 25)  # número de atributos a seleccionar
era = era.fit(X_train.values, y_train)
# imprimir resultados
atrib = era.support_
atributos = [columnas[i] for i in list(atrib.nonzero()[0])]
print(atributos)

['RANG_INGRESO', 'EDAD', 'ANTIGUEDAD', 'RANG_SDO_PASIVO_MENOS0', 'SDO_ACTIVO_MENOS0', 'SDO_ACTIVO_MENOS1', 'FLG_SEGURO_MENOS2', 'RANG_NRO_PRODUCTOS_MENOS0', 'FLG_NOMINA', 'NRO_ACCES_CANAL1_MENOS0', 'NRO_ACCES_CANAL1_MENOS1', 'NRO_ACCES_CANAL2_MENOS0', 'NRO_ACCES_CANAL2_MENOS1', 'NRO_ACCES_CANAL3_MENOS0', 'NRO_ACCES_CANAL3_MENOS1', 'NRO_ACCES_CANAL3_MENOS2', 'NRO_ACCES_CANAL3_MENOS3', 'NRO_ACCES_CANAL3_MENOS4', 'NRO_ACCES_CANAL3_MENOS5', 'NRO_ENTID_SSFF_MENOS0', 'NRO_ENTID_SSFF_MENOS1', 'NRO_ENTID_SSFF_MENOS2', 'NRO_ENTID_SSFF_MENOS3', 'NRO_ENTID_SSFF_MENOS4', 'NRO_ENTID_SSFF_MENOS5']


In [9]:
print(type(X_train), type(X_predic))
xx_train = X_train.loc[:,atributos].values
print(type(X_train), X_train.shape)
xx_predic = X_predic.loc[:,atributos].values
print(type(X_predic), X_predic.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'> (56262, 51)
<class 'pandas.core.frame.DataFrame'> (30000, 51)


In [10]:
print(type(xx_train), type(y_train))
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(kind='borderline2').fit_sample(xx_train, y_train)

print("X_smote:: ", X_smote.shape)
print("y_smote:: ", y_smote.shape)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
X_smote::  (98892, 25)
y_smote::  (98892,)


In [11]:
from sklearn.preprocessing import StandardScaler
X_smote = StandardScaler().fit_transform(X_smote.astype(float))
X_smote.shape

(98892, 25)

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=25)
#Construimos el modelo sobre los datos de entrenamiento
clf.fit(X_smote,y_smote)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [14]:
prediccion = clf.predict(xx_predic)
print(prediccion.shape)
prediccion_proba = clf.predict_proba(xx_predic)
print(prediccion_proba.shape)

(30000,)
(30000, 2)


In [18]:
print(prediccion)
print("//////////////")
print(prediccion_proba)
print("//////////////")
result = [float("{0:.2f}".format(_)) for _ in list(zip(*prediccion_proba))[1]]
print(result)


[1 1 1 ..., 1 1 1]
//////////////
[[ 0.    1.  ]
 [ 0.04  0.96]
 [ 0.2   0.8 ]
 ..., 
 [ 0.2   0.8 ]
 [ 0.    1.  ]
 [ 0.32  0.68]]
//////////////
[1.0, 0.96, 0.8, 0.88, 1.0, 0.84, 0.8, 0.56, 0.96, 0.76, 0.72, 0.92, 0.84, 1.0, 0.68, 1.0, 1.0, 0.96, 0.68, 0.64, 0.84, 0.96, 0.52, 0.84, 0.88, 0.88, 0.48, 1.0, 0.56, 0.48, 0.68, 1.0, 0.88, 0.72, 0.88, 0.56, 0.8, 0.68, 1.0, 0.76, 1.0, 1.0, 0.48, 0.88, 1.0, 1.0, 1.0, 0.92, 0.88, 1.0, 0.84, 0.92, 0.72, 1.0, 0.8, 0.92, 0.84, 0.56, 0.72, 0.64, 0.6, 0.96, 0.8, 1.0, 0.92, 1.0, 1.0, 0.64, 0.92, 0.8, 0.88, 0.84, 1.0, 1.0, 1.0, 1.0, 0.96, 0.72, 0.88, 1.0, 0.88, 0.96, 1.0, 0.6, 0.72, 1.0, 0.56, 0.88, 0.96, 0.88, 0.84, 1.0, 0.88, 0.84, 0.56, 0.68, 0.96, 0.92, 0.76, 0.96, 0.88, 0.68, 0.96, 1.0, 0.76, 0.92, 1.0, 1.0, 1.0, 0.92, 0.96, 0.92, 0.96, 1.0, 0.88, 1.0, 0.84, 0.96, 0.92, 0.88, 0.96, 0.76, 0.88, 0.88, 0.96, 0.44, 0.8, 0.8, 0.64, 0.72, 0.76, 0.6, 0.8, 0.84, 0.88, 0.96, 0.96, 0.76, 0.96, 1.0, 0.8, 0.64, 0.72, 1.0, 0.68, 0.76, 0.72, 0.72, 0.68, 0.84,

In [19]:
sum(prediccion)

29535

In [21]:
idd = BBVA_test['ID_CORRELATIVO']
idd.shape

(30000,)

In [22]:
resultados = list(zip(list(idd), result))
resultados

[(47411, 1.0),
 (39861, 0.96),
 (38898, 0.8),
 (50927, 0.88),
 (32969, 1.0),
 (89661, 0.84),
 (12197, 0.8),
 (71520, 0.56),
 (59759, 0.96),
 (36743, 0.76),
 (10736, 0.72),
 (5106, 0.92),
 (44506, 0.84),
 (47767, 1.0),
 (74686, 0.68),
 (63023, 1.0),
 (13584, 1.0),
 (317, 0.96),
 (62570, 0.68),
 (68244, 0.64),
 (72881, 0.84),
 (18280, 0.96),
 (70682, 0.52),
 (79902, 0.84),
 (35073, 0.88),
 (84331, 0.88),
 (70921, 0.48),
 (82245, 1.0),
 (17714, 0.56),
 (13993, 0.48),
 (658, 0.68),
 (33916, 1.0),
 (92737, 0.88),
 (22844, 0.72),
 (93339, 0.88),
 (29572, 0.56),
 (24751, 0.8),
 (37862, 0.68),
 (26676, 1.0),
 (8675, 0.76),
 (64245, 1.0),
 (57248, 1.0),
 (3531, 0.48),
 (9834, 0.88),
 (74246, 1.0),
 (56683, 1.0),
 (16847, 1.0),
 (96617, 0.92),
 (94167, 0.88),
 (42800, 1.0),
 (70106, 0.84),
 (42486, 0.92),
 (78076, 0.72),
 (85819, 1.0),
 (15786, 0.8),
 (61483, 0.92),
 (25630, 0.84),
 (22501, 0.56),
 (92447, 0.72),
 (13795, 0.64),
 (57101, 0.6),
 (55077, 0.96),
 (47600, 0.8),
 (5924, 1.0),
 (34900

In [23]:
import csv
resultados.insert(0, ('ID_CORRELATIVO', 'ATTRITION'))
resultados

[('ID_CORRELATIVO', 'ATTRITION'),
 (47411, 1.0),
 (39861, 0.96),
 (38898, 0.8),
 (50927, 0.88),
 (32969, 1.0),
 (89661, 0.84),
 (12197, 0.8),
 (71520, 0.56),
 (59759, 0.96),
 (36743, 0.76),
 (10736, 0.72),
 (5106, 0.92),
 (44506, 0.84),
 (47767, 1.0),
 (74686, 0.68),
 (63023, 1.0),
 (13584, 1.0),
 (317, 0.96),
 (62570, 0.68),
 (68244, 0.64),
 (72881, 0.84),
 (18280, 0.96),
 (70682, 0.52),
 (79902, 0.84),
 (35073, 0.88),
 (84331, 0.88),
 (70921, 0.48),
 (82245, 1.0),
 (17714, 0.56),
 (13993, 0.48),
 (658, 0.68),
 (33916, 1.0),
 (92737, 0.88),
 (22844, 0.72),
 (93339, 0.88),
 (29572, 0.56),
 (24751, 0.8),
 (37862, 0.68),
 (26676, 1.0),
 (8675, 0.76),
 (64245, 1.0),
 (57248, 1.0),
 (3531, 0.48),
 (9834, 0.88),
 (74246, 1.0),
 (56683, 1.0),
 (16847, 1.0),
 (96617, 0.92),
 (94167, 0.88),
 (42800, 1.0),
 (70106, 0.84),
 (42486, 0.92),
 (78076, 0.72),
 (85819, 1.0),
 (15786, 0.8),
 (61483, 0.92),
 (25630, 0.84),
 (22501, 0.56),
 (92447, 0.72),
 (13795, 0.64),
 (57101, 0.6),
 (55077, 0.96),
 (

In [24]:
with open("output_proba_2.csv", 'w') as resultFile:
       wr = csv.writer(resultFile, dialect='excel')
       for row in resultados:
           wr.writerow(row)